In [1]:
import os
import typing as t
import numpy as np
from skimage import measure
from skimage.morphology import dilation, ball, diamond
from skimage.measure import label, regionprops
import numba
import numba.typed
from sklearn.neighbors import KDTree
import functools
from copy import copy
import itertools
import pandas as pd
import numpy as np
import skimage.morphology
import skimage.color
import scipy.stats
from aicsimageio import AICSImage, imread
from aicsimageio.writers import OmeTiffWriter
import matplotlib.pyplot as plt

In [2]:
def cc_centroids(mask):
    if mask.dtype == bool:
        mask = label(mask)
    ccs = regionprops(mask)
    centroids, labels = [], []
    for cc in ccs:
        centroids.append(cc.centroid)
        labels.append(cc.label)
    return np.array(centroids), np.array(labels)


def coordinates_to_mask(points: np.ndarray,
                        shape: t.Optional[t.Tuple] = None) -> np.ndarray:

  
    points = points.astype(np.int64)
    dim_max = tuple([points[:, i].max()+1 for i in range(points.shape[1])])
    if shape is None:
        shape = dim_max
    else:
        assert len(shape) == points.shape[1]
        shape = tuple([shape[i] or dim_max[i] for i in range(points.shape[1])])
    arr = np.zeros(shape, dtype=bool)
    ix = tuple(points[:, d] for d in range(points.shape[1]))
    arr[ix] = True
    return arr


def merge_close_points_cc(points,
                          radius =2) :

    assert points.shape[1] in {2, 3}
    radius -= 1
    mask = coordinates_to_mask(points)
    if radius > 0:
        if points.shape[1] == 2:
            se = diamond(radius)
        else:
            se = ball(radius)
        mask = dilation(mask, se)
    centroids, _ = cc_centroids(mask)
    return centroids

def expand_N(seq, bases="ACGT"):
    """Expand N in seq to ACGT

    >>> expand_N("AN")
    ['AA', 'AC', 'AG', 'AT']
    """
    res = []
    N_pos = [i for i in range(len(seq)) if seq[i] == "N"]
    perm = itertools.product(bases, repeat=len(N_pos))
    for s in perm:
        seq_rep = list(seq)
        for i, p in enumerate(N_pos):
            seq_rep[p] = s[i]
        res.append("".join(seq_rep))
    return res

def extract_sub_mats(
        img, points,
        sub_shape, border_value = 0) :

    points = points.astype(np.int64)
    assert len(sub_shape) in (2, 3)
    # add border to image
    radius = [d // 2 for d in sub_shape]
    shape_with_border = [
        d + 2*radius[i]
        for i, d in enumerate(img.shape)
    ]
    img_with_border = np.full(
        tuple(shape_with_border),
        border_value, dtype=img.dtype
    )
    slices = [
        slice(radius[i], radius[i] + d)
        for i, d in enumerate(img.shape)
    ]
    img_with_border[tuple(slices)] = img
    # extract sub matrices
    radius = numba.typed.List(radius)
    if len(radius) == 2:
        subs = _extract_sub_mat_2d(img_with_border, points, radius)
    else:
        subs = _extract_sub_mat_3d(img_with_border, points, radius)
    sub_matrices = np.array(subs)
    return sub_matrices


@numba.jit(nopython=True)
def _extract_sub_mat_2d(img_with_border, points, radius):
    subs = []
    for i in range(points.shape[0]):
        y = points[i, 0]
        x = points[i, 1]
        r0 = radius[0]
        r1 = radius[1]
        sub = img_with_border[
            y:y+2*r0+1,
            x:x+2*r1+1,
        ]
        subs.append(sub)
    return subs


@numba.jit(nopython=True)
def _extract_sub_mat_3d(img_with_border, points, radius):
    subs = []
    for i in range(points.shape[0]):
        z = points[i, 0]
        y = points[i, 1]
        x = points[i, 2]
        r0 = radius[0]
        r1 = radius[1]
        r2 = radius[2]
        sub = img_with_border[
            z:z+2*r0+1,
            y:y+2*r1+1,
            x:x+2*r2+1
        ]
        subs.append(sub)
    return subs


def assign_spots(
        spots,
        mask,
        dist_th,
        max_iter=0,
        iter_dist_step=10,
        ):

    assert len(mask.shape) in (2, 3)
    centers, labels = cc_centroids(mask)
    assert centers.shape[1] == len(mask.shape)
    search_map = np.zeros(  # label(index) -> center(value)
        (labels.max(), centers.shape[1]), dtype=centers.dtype)
    search_map[labels - 1] = centers
    pos_each_axes = np.where(mask > 0)
    pos_ = np.c_[pos_each_axes]
    tree = KDTree(pos_)
    dist, idx = tree.query(spots)
    dist, idx = np.concatenate(dist), np.concatenate(idx)
    clost = pos_[idx, :]
    if centers.shape[1] == 2:
        mask_val = mask[clost[:, 0], clost[:, 1]]
    else:
        mask_val = mask[clost[:, 0], clost[:, 1], clost[:, 2]]
    res = search_map[mask_val - 1]
    res[dist > dist_th, :] = np.nan

    # Iteratively find neighbors
    for _ in range(max_iter):
        nan_idxs = np.where(np.isnan(res))[0]
        if nan_idxs.shape[0] == 0:
            break
        non_nan_idxs = np.where(~np.isnan(res))[0]
        non_nan_res = res[non_nan_idxs]
        non_nan_pos = spots[non_nan_idxs, :]
        tree = KDTree(non_nan_pos)
        nan_pos = spots[nan_idxs, :]
        dist, idx = tree.query(nan_pos)
        dist, idx = np.concatenate(dist), np.concatenate(idx)
        nan_res = non_nan_res[idx]
        nan_res[dist > iter_dist_step] = np.nan
        res[nan_idxs] = nan_res

    return res
def ReadImage(path):
    img = AICSImage(path)
    return img
def tophat_extrema(
        img,
        q,  # type: ignore
        ):
    t = skimage.morphology.white_tophat(img)
    h = np.quantile(t, q/100)
    spots = np.c_[np.where(skimage.morphology.extrema.h_maxima(t, h))]
    return spots
def merge_spots(
        coords, radius  # type: ignore
        ) :  # noqa
    res = merge_close_points_cc(coords, radius)
    return res
def extract_fetures(
        img, coords,
        sub_shape= "None",
        ) :  # noqa

    sub_shape = eval(sub_shape)
    c = coords
    im = img
    if sub_shape is None:
        features = im[:, c[:, 0], c[:, 1], c[:, 2]]
    else:
        features = np.array([
            extract_sub_mats(img[ixch], coords, sub_shape)
            for ixch in range(img.shape[0])
        ])
    return features
def parse_one_cycle(
        feature_vec, channel_num= 2,
        ):  # type: ignore # noqa 

    assert channel_num in (1, 2)
    feature_vec = feature_vec.astype(np.float32)
    V = np.sort(feature_vec, axis=0)
    ind = np.argsort(feature_vec, axis=0)
    ratios_each_nch = []
    for nch in range(1, channel_num + 1):
        # iter over possible channel number
        # calculate probability of this channel number
        s_ = V[-nch:, :].min(axis=0)
        n_ = V[:-nch, :].max(axis=0)
        r_ = (s_ + 1) / (n_ + 1)
        ratios_each_nch.append(r_)
    if channel_num == 1:
        ratio = r_
        channel_idx = ind[-1, :][np.newaxis, :]
    else:
        # calculate the most possible channel number
        ratios_each_nch = np.stack(ratios_each_nch)
        nch_max = np.argmax(ratios_each_nch, axis=0)
        channel_idx: np.ndarray = np.where(
            nch_max == 0,
            np.tile(ind[-1, :], (channel_num, 1)),
            ind[-channel_num:, :]
        )
        channel_idx.sort(axis=0)
        ratio = np.choose(nch_max, ratios_each_nch)
    e_ = np.exp(ratio)
    e_[e_ == np.inf] = 1e10
    prob = (e_ - 1) / (e_ + 1)
    return channel_idx, prob
def decode(
        channel_idx, code_to_gene,
        channel_order = "ACGT") :  # type: ignore

    seq_order = np.array(list(channel_order))
    channel_seq = seq_order[channel_idx]
    df_ = pd.DataFrame(channel_seq).T
    seq = df_.sum(axis=1)
    # fill N in code_to_gene
    filled = copy(code_to_gene)
    for code, gene in code_to_gene.items():
        if 'N' not in code:
            continue
        filled.pop(code)
        for exp_code in expand_N(code):
            filled[exp_code] = gene
    # set not exist gene to unknow
    for c in seq.unique():
        if c not in filled:
            filled[c] = "unknow"
    df_c2g = pd.DataFrame(filled.items(), columns=['code', 'gene'])
    df_c2g.index = df_c2g['code']
    gene = df_c2g.loc[seq]['gene']
    return gene.values, gene.index.values

def gene_table(
        coords, probs,
        decoded, codes,
        ) :  # noqa
    """Construct gene result table"""
    columns = {}
    for col in range(coords.shape[1]):
        columns[f'dim_{col}'] = coords[:, col]
    columns.update({
        'gene': decoded,
        'codes': codes,
    })
    if not isinstance(probs, list):
        probs = [probs]
    for col, prob in enumerate(probs):
        columns[f'probability_{col}'] = prob
    return pd.DataFrame(columns)

def GetImageData(img):
    return img.data

def gene_to_cell(
        gene_table,
        cell_mask,
        dist_thresh= 30.0,
        max_iter= 5,
        dist_step= 10.0,
        ) :  # type: ignore

    dims = [c for c in gene_table.columns if c.startswith('dim')]
    spots = gene_table[dims].values
    related_centers = assign_spots(
        spots, cell_mask, dist_thresh,
        max_iter=max_iter, iter_dist_step=dist_step)
    df_center = pd.DataFrame({
        d: related_centers[:, i]
        for i, d in enumerate(dims)
    }).applymap("{:.2f}".format)
    center_ser = "(" + df_center.iloc[:, 0]
    for i in range(1, len(dims)):
        center_ser += (", " + df_center.iloc[:, i])
    center_ser += ")"
    df = pd.DataFrame({
        'gene': gene_table['gene'].values,
        'cell': center_ser.values
    })
    counts = df.value_counts()
    exp_mat = counts.unstack(fill_value=0).T
    return exp_mat, related_centers



In [3]:
def normalize_with_spots(
        img,
        spots,
        ):  # type: ignore  # noqa
    new_channels = []
    means = []
    for idx, s in enumerate(spots):
        sub_img = img[idx]
        if len(sub_img.shape) == 3:
            m = sub_img[s[:, 0], s[:, 1], s[:, 2]].mean()
        else:
            assert len(sub_img.shape) == 2
            m = sub_img[s[:, 0], s[:, 1]].mean()
        means.append(m)
        new_channels.append(sub_img / m)
    new_img = np.stack(new_channels)
    return new_img, means

def predict_spot(it):
    pred_spots, enh_img = ufish.predict(it,axes="zyx",blend_3d=False,intensity_threshold=0.1)
    return pred_spots.values

def nomorizepic(path):
    it=GetImageData(ReadImage(path)) 
    it=it[0][0][:4].max(axis=1)
    pred_spots1 = tophat_extrema(it[0],q=99.50)
    pred_spots2 = tophat_extrema(it[1],q=99.50)
    pred_spots3 = tophat_extrema(it[2],q=99.50)
    pred_spots4 = tophat_extrema(it[3],q=99.50)
    listout=[pred_spots1,pred_spots2,pred_spots3,pred_spots4]
    it,_ = normalize_with_spots(it,listout)
    return it

def WriteTiff(img, out_path, dim_order = 'CZYX') :
    OmeTiffWriter.save(img, out_path, dim_order=dim_order)
    return out_path

def tophat_extrema(img: np.ndarray,q):  # noqa
    t = skimage.morphology.white_tophat(img)
    h = np.quantile(t, q/100)
    spots = np.c_[np.where(skimage.morphology.extrema.h_maxima(t, h))]
    return spots

def analyze(path,dic,q,sub_shape):
    it=GetImageData(ReadImage(path))
    print(it.shape)  
    it=it.reshape(it.shape[2],it.shape[3],it.shape[4],it.shape[5])
    it1=it[:4].max(axis=0)
    it2=it[:4]
    pred_spots = tophat_extrema(it1,q=q)
    output=merge_spots(pred_spots,1)
    output=output.astype(int)
    # fetures=extract_fetures(it2,output,sub_shape="(3, 3, 3)")
    # fetures=fetures.mean(axis=(2,3,4))
    if sub_shape == 'None':
        fetures=extract_fetures(it2,output,sub_shape=sub_shape)
    else:
        fetures=extract_fetures(it2,output,sub_shape=sub_shape)
        fetures=fetures.mean(axis=(2,3,4))
    channel_idx, prob=parse_one_cycle(fetures,2)
    decoded,codes = decode(channel_idx,dic)
    table=gene_table(output,prob,decoded,codes)
    return table


In [4]:
def mask2gray(img_path):
    mask = imread(img_path)
    mask=mask[0][0][0][0]
    mask = np.where(mask != 0, 255, 0)
    rgba_image = color.label2rgb(mask, bg_color=(1, 1, 1, 0), bg_label=0)
    gray_image = color.rgb2gray(rgba_image)
    return gray_image

In [5]:
from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())
samples = ['C2-1']

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'CD163': '#EE5061', 'MRC1': '#CA2D46', 'CD3E': '#0072D4', 'CD8A': '#428CF2', 
              'CD4': '#00BDD9', 'FXOP3': '#B0E7FF', 'BANK1': '#767E00', 'CD19': '#C35D14' , 'MPO': '#FFA7FF',
              'CD94': '#757FAF', 'CD14': '#A70063', 'CD11c': '#F9F871', 'SLA-DRB1' : '#F2FEDC',
		      'CD31' : '#FF9A6B','RTKN2' : '#296218', 'SFTPB' : '#5ABAB6',
              'RPST': '#0099e5', 'TMEM107': '#ff4c4c', 'MX1': '#34bf49', 'TNFAIP3': '#00a98f', 
              'COX1': '#be0027', 'STAT1': '#cf8d2e', 'DDIT3': '#e4e932', 'PIK3R1': '#371777' , 'DUSP1': '#52325d',
              'PYCARD': '#511378', 'IL8': '#8a7967', 'IL10': '#a626aa', 'TNFSF10': '#88aca1', 'CXCL10' : '#788cb6', 'TFNA' : '#D8A59E',
		      'PDL1' : '#00aee6','CD279' : '#0169a4', 'IL17A' : '#f1632a', 'TGFB': '#2baf2b',  'IL1B2':'#fdbd10',
            'CCL2':'#d74e26', 'IFNA':'#ffc2e5', 'IFNB':'#c1f1fc', 'IFNG':'#ffaaaa', 'CSF3':'#ee70a6', 'TNFSF13B':'#373b9c', 
            'CD1D':'#b6acd1'}
    
for sample in samples:

    # os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    # for gene,colors in dict_color.items():
    #     fig,ax = plt.subplots(figsize=figsize)
    #     ax.imshow(img, cmap='gray', alpha=0.1)
    #     df_gene = df[df["gene"] == gene]
    #     y = df_gene["dim_1"]
    #     x = df_gene["dim_2"]
    #     plt.scatter(x, y, s=50, c=colors, label=gene)
    #     plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #     fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
    #     plt.cla()
    #     plt.close(fig)
    #     print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        ax_2.scatter(x, y, s=20, c=colors, label=gene, alpha=0.70)
    ax_2.legend(prop={"size":10}, loc=2, ncol=1, bbox_to_anchor=(1.05,1.0), borderaxespad=0.)
    #plt.show()
    ax_2.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False)    # ax_2.axis('off')  # Do not display x and y axes

    fig_2.savefig(f"./{sample}-all_gene.png", format="png")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1966, 1943)
C2-1 plot over


In [10]:
from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())
samples = ['C3-3', 'C3-2']

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'CD163': '#EE5061', 'MRC1': '#CA2D46', 'CD3E': '#0072D4', 'CD8A': '#428CF2', 
              'CD4': '#00BDD9', 'FXOP3': '#B0E7FF', 'BANK1': '#767E00', 'CD19': '#C35D14' , 'MPO': '#FFA7FF',
              'CD94': '#757FAF', 'CD14': '#A70063', 'CD11c': '#F9F871', 'SLA-DRB1' : '#F2FEDC',
		      'CD31' : '#FF9A6B','RTKN2' : '#296218', 'SFTPB' : '#5ABAB6',
              'RPST': '#0099e5', 'TMEM107': '#ff4c4c', 'MX1': '#34bf49', 'TNFAIP3': '#00a98f', 
              'COX1': '#be0027', 'STAT1': '#cf8d2e', 'DDIT3': '#e4e932', 'PIK3R1': '#371777' , 'DUSP1': '#52325d',
              'PYCARD': '#511378', 'IL8': '#8a7967', 'IL10': '#a626aa', 'TNFSF10': '#88aca1', 'CXCL10' : '#788cb6', 'TFNA' : '#D8A59E',
		      'PDL1' : '#00aee6','CD279' : '#0169a4', 'IL17A' : '#f1632a', 'TGFB': '#2baf2b',  'IL1B2':'#fdbd10',
            'CCL2':'#d74e26', 'IFNA':'#ffc2e5', 'IFNB':'#c1f1fc', 'IFNG':'#ffaaaa', 'CSF3':'#ee70a6', 'TNFSF13B':'#373b9c', 
            'CD1D':'#b6acd1'}
    
for sample in samples:

    # os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    # for gene,colors in dict_color.items():
    #     fig,ax = plt.subplots(figsize=figsize)
    #     ax.imshow(img, cmap='gray', alpha=0.1)
    #     df_gene = df[df["gene"] == gene]
    #     y = df_gene["dim_1"]
    #     x = df_gene["dim_2"]
    #     plt.scatter(x, y, s=50, c=colors, label=gene)
    #     plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #     fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
    #     plt.cla()
    #     plt.close(fig)
    #     print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        ax_2.scatter(x, y, s=20, c=colors, label=gene, alpha=0.70)
    ax_2.legend(prop={"size":10}, loc=2, ncol=1, bbox_to_anchor=(1.05,1.0), borderaxespad=0.)
    #plt.show()
    ax_2.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False)    # ax_2.axis('off')  # Do not display x and y axes

    fig_2.savefig(f"./{sample}-all_gene.png", format="png")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1953, 1946)
C3-3 plot over
(1979, 1948)
C3-2 plot over


In [11]:
samples = ['A4-3']

from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'CD163': '#EE5061', 'MRC1': '#CA2D46', 'CD3E': '#0072D4', 'CD8A': '#428CF2', 
              'CD4': '#00BDD9', 'FXOP3': '#B0E7FF', 'BANK1': '#767E00', 'CD19': '#C35D14' , 'MPO': '#FFA7FF',
              'CD94': '#757FAF', 'CD14': '#A70063', 'CD11c': '#F9F871', 'SLA-DRB1' : '#F2FEDC',
		      'CD31' : '#FF9A6B','RTKN2' : '#296218', 'SFTPB' : '#5ABAB6',
              'RPST': '#0099e5', 'TMEM107': '#ff4c4c', 'MX1': '#34bf49', 'TNFAIP3': '#00a98f', 
              'COX1': '#be0027', 'STAT1': '#cf8d2e', 'DDIT3': '#e4e932', 'PIK3R1': '#371777' , 'DUSP1': '#52325d',
              'PYCARD': '#511378', 'IL8': '#8a7967', 'IL10': '#a626aa', 'TNFSF10': '#88aca1', 'CXCL10' : '#788cb6', 'TFNA' : '#D8A59E',
		      'PDL1' : '#00aee6','CD279' : '#0169a4', 'IL17A' : '#f1632a', 'TGFB': '#2baf2b',  'IL1B2':'#fdbd10',
            'CCL2':'#d74e26', 'IFNA':'#ffc2e5', 'IFNB':'#c1f1fc', 'IFNG':'#ffaaaa', 'CSF3':'#ee70a6', 'TNFSF13B':'#373b9c', 
            'CD1D':'#b6acd1', 'B646L':'#7F1684', 'MGF_360-9L':'#CC65CF', 'MGF_360-2L':'#CB9CCA', 
            'MGF_505-7R':'#514EB6', 'MGF_110-9L':'#978BFC', 'EP402R':'#DECDFF', 
            'S273R':'#FFC1D6'}
    
for sample in samples:

    # os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    # for gene,colors in dict_color.items():
    #     fig,ax = plt.subplots(figsize=figsize)
    #     ax.imshow(img, cmap='gray', alpha=0.1)
    #     df_gene = df[df["gene"] == gene]
    #     y = df_gene["dim_1"]
    #     x = df_gene["dim_2"]
    #     plt.scatter(x, y, s=50, c=colors, label=gene)
    #     plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #     fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
    #     plt.cla()
    #     plt.close(fig)
    #     print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        ax_2.scatter(x, y, s=20, c=colors, label=gene, alpha=0.70)
    ax_2.legend(prop={"size":10}, loc=2, ncol=1, bbox_to_anchor=(1.05,1.0), borderaxespad=0.)
    #plt.show()
    ax_2.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False)    # ax_2.axis('off')  # Do not display x and y axes

    fig_2.savefig(f"./{sample}-all_gene.png", format="png")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1987, 1944)
A4-3 plot over


In [15]:
from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())
samples = ['C6-3']

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'CD163': '#EE5061', 'MRC1': '#CA2D46', 'CD3E': '#0072D4', 'CD8A': '#428CF2', 
              'CD4': '#00BDD9', 'FXOP3': '#B0E7FF', 'BANK1': '#767E00', 'CD19': '#C35D14' , 'MPO': '#FFA7FF',
              'CD94': '#757FAF', 'CD14': '#A70063', 'CD11c': '#F9F871', 'SLA-DRB1' : '#F2FEDC',
		      'CD31' : '#FF9A6B','RTKN2' : '#296218', 'SFTPB' : '#5ABAB6',
              'RPST': '#0099e5', 'TMEM107': '#ff4c4c', 'MX1': '#34bf49', 'TNFAIP3': '#00a98f', 
              'COX1': '#be0027', 'STAT1': '#cf8d2e', 'DDIT3': '#e4e932', 'PIK3R1': '#371777' , 'DUSP1': '#52325d',
              'PYCARD': '#511378', 'IL8': '#8a7967', 'IL10': '#a626aa', 'TNFSF10': '#88aca1', 'CXCL10' : '#788cb6', 'TFNA' : '#D8A59E',
		      'PDL1' : '#00aee6','CD279' : '#0169a4', 'IL17A' : '#f1632a', 'TGFB': '#2baf2b',  'IL1B2':'#fdbd10',
            'CCL2':'#d74e26', 'IFNA':'#ffc2e5', 'IFNB':'#c1f1fc', 'IFNG':'#ffaaaa', 'CSF3':'#ee70a6', 'TNFSF13B':'#373b9c', 
            'CD1D':'#b6acd1'}
    
for sample in samples:

    # os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    # for gene,colors in dict_color.items():
    #     fig,ax = plt.subplots(figsize=figsize)
    #     ax.imshow(img, cmap='gray', alpha=0.1)
    #     df_gene = df[df["gene"] == gene]
    #     y = df_gene["dim_1"]
    #     x = df_gene["dim_2"]
    #     plt.scatter(x, y, s=50, c=colors, label=gene)
    #     plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #     fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
    #     plt.cla()
    #     plt.close(fig)
    #     print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=20, c=colors, label=gene, alpha=0.70)
    plt.legend(prop={"size":10}, loc=2, ncol = 1, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #plt.show()
    fig_2.savefig(f"./{sample}-all_gene.pdf", format="pdf")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1935, 1942)
C6-3 plot over


In [13]:
samples = ['A2-4']

from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'CD163': '#EE5061', 'MRC1': '#CA2D46', 'CD3E': '#0072D4', 'CD8A': '#428CF2', 
              'CD4': '#00BDD9', 'FXOP3': '#B0E7FF', 'BANK1': '#767E00', 'CD19': '#C35D14' , 'MPO': '#FFA7FF',
              'CD94': '#757FAF', 'CD14': '#A70063', 'CD11c': '#F9F871', 'SLA-DRB1' : '#F2FEDC',
		      'CD31' : '#FF9A6B','RTKN2' : '#296218', 'SFTPB' : '#5ABAB6',
              'RPST': '#0099e5', 'TMEM107': '#ff4c4c', 'MX1': '#34bf49', 'TNFAIP3': '#00a98f', 
              'COX1': '#be0027', 'STAT1': '#cf8d2e', 'DDIT3': '#e4e932', 'PIK3R1': '#371777' , 'DUSP1': '#52325d',
              'PYCARD': '#511378', 'IL8': '#8a7967', 'IL10': '#a626aa', 'TNFSF10': '#88aca1', 'CXCL10' : '#788cb6', 'TFNA' : '#D8A59E',
		      'PDL1' : '#00aee6','CD279' : '#0169a4', 'IL17A' : '#f1632a', 'TGFB': '#2baf2b',  'IL1B2':'#fdbd10',
            'CCL2':'#d74e26', 'IFNA':'#ffc2e5', 'IFNB':'#c1f1fc', 'IFNG':'#ffaaaa', 'CSF3':'#ee70a6', 'TNFSF13B':'#373b9c', 
            'CD1D':'#b6acd1', 'B646L':'#7F1684', 'MGF_360-9L':'#CC65CF', 'MGF_360-2L':'#CB9CCA', 
            'MGF_505-7R':'#514EB6', 'MGF_110-9L':'#978BFC', 'EP402R':'#DECDFF', 
            'S273R':'#FFC1D6'}
    
for sample in samples:

    # os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    # for gene,colors in dict_color.items():
    #     fig,ax = plt.subplots(figsize=figsize)
    #     ax.imshow(img, cmap='gray', alpha=0.1)
    #     df_gene = df[df["gene"] == gene]
    #     y = df_gene["dim_1"]
    #     x = df_gene["dim_2"]
    #     plt.scatter(x, y, s=50, c=colors, label=gene)
    #     plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #     fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
    #     plt.cla()
    #     plt.close(fig)
    #     print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=20, c=colors, label=gene, alpha=0.70)
    plt.legend(prop={"size":10}, loc=2, ncol = 1, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #plt.show()
    fig_2.savefig(f"./{sample}-all_gene.pdf", format="pdf")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1923, 1945)
A2-4 plot over


In [9]:
samples = ['A6-3']

from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'B646L':'#7F1684', 'MGF_360-9L':'#CC65CF', 'MGF_360-2L':'#CB9CCA', 
            'MGF_505-7R':'#514EB6', 'MGF_110-9L':'#978BFC', 'EP402R':'#DECDFF', 
            'S273R':'#FFC1D6'}

# dict_color = {'B646L':'#7F1684'}

for sample in samples:

    # os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    # for gene,colors in dict_color.items():
    #     fig,ax = plt.subplots(figsize=figsize)
    #     ax.imshow(img, cmap='gray', alpha=0.1)
    #     df_gene = df[df["gene"] == gene]
    #     y = df_gene["dim_1"]
    #     x = df_gene["dim_2"]
    #     plt.scatter(x, y, s=50, c=colors, label=gene)
    #     plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #     fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
    #     plt.cla()
    #     plt.close(fig)
    #     print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    # ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, -y, s=20, c=colors, label=gene, alpha=0.70)
    plt.legend(prop={"size":10}, loc=2, ncol = 1, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #plt.show()
    fig_2.savefig(f"./{sample}-virus.pdf", format="pdf")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1970, 1945)
A6-3 plot over


In [4]:
samples = ["C3-1", "C3-2", "C3-3", "C2-1", "C2-2", "C2-3","C4-1", "C4-2", "C4-3", "C5-1", "C5-2", "C5-3", "C6-1", "C6-2", "C6-3"]

In [4]:
samples = ["C2-1"]
from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'CD163': '#FA4E27', 'MRC1': '#FA4E27', 'CD3E': '#F9F871', 'CD8A': '#F9F871', 
              'CD4': '#F9F871', 'FXOP3': '#F9F871', 'BANK1': '#007FFF', 'CD19': '#007FFF' , 'MPO': '#4C4452',
              'IL8': '#4C4452', 'CD94': '#FA4E27', 'CD14': '#AB8800', 'CD11c': '#007B66', 'SLA-DRB1' : '#007B66',
		      'CD31' : '#AB8800','RTKN2' : '#FF6B97', 'SFTPB' : '#FF9A6B'}

def mask2gray(img_path):
    mask = imread(img_path)
    mask=mask[0][0][0][0]
    mask = np.where(mask != 0, 255, 0)
    rgba_image = color.label2rgb(mask, bg_color=(1, 1, 1, 0), bg_label=0)
    gray_image = color.rgb2gray(rgba_image)
    return gray_image
    
for sample in samples:

    os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    for gene,colors in dict_color.items():
        fig,ax = plt.subplots(figsize=figsize)
        ax.imshow(img, cmap='gray', alpha=0.1)
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=50, c=colors, label=gene)
        plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
        fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
        plt.cla()
        plt.close(fig)
        print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=35, c=colors, label=gene, alpha=0.70)
    plt.legend(prop={"size":15}, loc=2, ncol = 1, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #plt.show()
    fig_2.savefig(f"./scatterplot/{sample}-all_gene.pdf", format="pdf")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1966, 1943)
C2-1-CD163 plot over
C2-1-MRC1 plot over
C2-1-CD3E plot over
C2-1-CD8A plot over
C2-1-CD4 plot over
C2-1-FXOP3 plot over
C2-1-BANK1 plot over
C2-1-CD19 plot over
C2-1-MPO plot over
C2-1-IL8 plot over
C2-1-CD94 plot over
C2-1-CD14 plot over
C2-1-CD11c plot over
C2-1-SLA-DRB1 plot over
C2-1-CD31 plot over
C2-1-RTKN2 plot over
C2-1-SFTPB plot over
C2-1 plot over


In [6]:
# samples = ["A2-1", "A2-2", "A2-3", "A2-4", "A3-1", "A3-2","A3-3", "A4-1","A4-2", "A4-3", "A5-1", "A5-2", "A5-3", "A6-1", "A6-2", "A6-3", "A6-4"]
samples = ["A2-2"]

from skimage import color

dict_color = {'CD163': '#20a994', 'MRC1': '#966d40', 'CD3E': '#acbf73', 'CD8A': '#acbf73', 
              'CD4': '#acbf73', 'FXOP3': '#acbf73', 'BANK1': '#bda7a5', 'CD19': '#bda7a5' , 'MPO': '#9fa8d5',
              'IL8': '#9fa8d5', 'CD94': '#3db37d', 'CD14': '#ce1eb0', 'CD11c': '#410ec7', 'SLA-DRB1' : '#410ec7',
		      'CD31' : '#DAA520','RTKN2' : '#A52A2A', 'SFTPB' : '#90EE90', 'B646L': '#BA55D3',  'MGF_110-9L':'#BA55D3',
                'MGF_360-9L':'#BA55D3', 'MGF_360-2L':'#BA55D3', 'MGF_505-7R':'#BA55D3', 'EP402R':'#BA55D3', 
              'S273R':'#BA55D3'}

def mask2gray(img_path):
    mask = imread(img_path)
    mask=mask[0][0][0][0]
    mask = np.where(mask != 0, 255, 0)
    rgba_image = color.label2rgb(mask, bg_color=(1, 1, 1, 0), bg_label=0)
    gray_image = color.rgb2gray(rgba_image)
    return gray_image
    
for sample in samples:

    os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    for gene,colors in dict_color.items():
        fig,ax = plt.subplots(figsize=figsize)
        ax.imshow(img, cmap='gray', alpha=0.1)
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=50, c=colors, label=gene)
        plt.legend(prop={"size":100}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
        fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.png", format="png")
        plt.cla()
        plt.close(fig)
        print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=35, c=colors, label=gene, alpha=0.70)
    plt.legend(prop={"size":15}, loc=2, ncol = 1, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #plt.show()
    fig_2.savefig(f"./scatterplot/{sample}-all_gene.png", format="png")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1916, 1948)
A2-2-CD163 plot over
A2-2-MRC1 plot over
A2-2-CD3E plot over
A2-2-CD8A plot over
A2-2-CD4 plot over
A2-2-FXOP3 plot over
A2-2-BANK1 plot over
A2-2-CD19 plot over
A2-2-MPO plot over
A2-2-IL8 plot over
A2-2-CD94 plot over
A2-2-CD14 plot over
A2-2-CD11c plot over
A2-2-SLA-DRB1 plot over
A2-2-CD31 plot over
A2-2-RTKN2 plot over
A2-2-SFTPB plot over
A2-2-B646L plot over
A2-2-MGF_110-9L plot over
A2-2-MGF_360-9L plot over
A2-2-MGF_360-2L plot over
A2-2-MGF_505-7R plot over
A2-2-EP402R plot over
A2-2-S273R plot over
A2-2 plot over


In [7]:
# samples = ["C2-1","A2-1", "A2-2", "A2-3", "A2-4", "A3-1", 
#            "A3-2","A3-3", "A4-1","A4-2", "A4-3", "A5-1",
#          "A5-2", "A5-3", "A6-1", "A6-2", "A6-3", "A6-4", 
#          "C3-1", "C3-2", "C3-3", "C2-2", "C2-3",
#          "C4-1", "C4-2", "C4-3", "C5-1", "C5-2", "C5-3", 
#          "C6-1", "C6-2", "C6-3"]

samples = ["A2-2"]
for sample in samples:
    name=f"./spots/{sample}.csv"
    con = pd.read_csv(name)
    con = con.iloc[:,:3]
    con.dropna(inplace=True)
    
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    path2=f"./mask/{dir}_{num}_mask.tif"
    mask=GetImageData(ReadImage(path2))
    print(mask.shape)
    mask=mask[0][0][0][0]
    gene2cell,related_ct=gene_to_cell(con,mask)
    con["0"]=related_ct[:,0]
    con["1"]=related_ct[:,1]
    con.dropna(subset=['0'],inplace=True)
    con.to_csv(f'{sample}-cellspots.csv',index=False)
    gene2cell.to_csv(f"./matrix/{sample}-gene_to_cell.csv")

    # print(con)
    gene_counts = con['gene'].value_counts()
    gene_counts.to_csv(f'./spots/{sample}-gene_counts.txt', header=True, sep='\t')
    print(f'count {sample} over')
    # print(gene_counts)
    #tablef=analyze(path06,dic6)
  

(1, 1, 1, 1, 1916, 1948)


C:\Users\woikaka\AppData\Local\Temp\ipykernel_300\725725157.py:298: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  }).applymap("{:.2f}".format)


count A2-2 over


In [12]:
samples = ["C3-1", "C3-2", "C3-3", "C2-1", "C2-2", "C2-3","C4-1", "C4-2", "C4-3", "C5-1", "C5-2", "C5-3", "C6-1", "C6-2", "C6-3"]

from skimage import color
# os.chdir(f'E:\project\gh\images')
# print(os.getcwd())

spots_dir = './spots'
os.makedirs(f"./scatterplot", exist_ok=True)

dict_color = {'CD163': '#EE5061', 'MRC1': '#EE5061', 'CD3E': '#0072D4', 'CD8A': '#0072D4', 
              'CD4': '#0072D4', 'FXOP3': '#0072D4', 'BANK1': '#C35D14', 'CD19': '#C35D14' , 'MPO': '#FFA7FF',
              'IL8': '#FFA7FF', 'CD94': '#757FAF', 'CD14': '#A70063', 'CD11c': '#F9F871', 'SLA-DRB1' : '#F9F871',
		      'CD31' : '#FF9A6B','RTKN2' : '#296218', 'SFTPB' : '#5ABAB6'}

def mask2gray(img_path):
    mask = imread(img_path)
    mask=mask[0][0][0][0]
    mask = np.where(mask != 0, 255, 0)
    rgba_image = color.label2rgb(mask, bg_color=(1, 1, 1, 0), bg_label=0)
    gray_image = color.rgb2gray(rgba_image)
    return gray_image
    
for sample in samples:

    os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}-cellspots.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    for gene,colors in dict_color.items():
        fig,ax = plt.subplots(figsize=figsize)
        ax.imshow(img, cmap='gray', alpha=0.1)
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=50, c=colors, label=gene)
        plt.legend(prop={"size":10}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
        fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
        plt.cla()
        plt.close(fig)
        print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=35, c=colors, label=gene, alpha=0.70)
    plt.legend(prop={"size":15}, loc=2, ncol = 1, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #plt.show()
    fig_2.savefig(f"./scatterplot/{sample}-all_gene.pdf", format="pdf")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1975, 1944)
C3-1-CD163 plot over
C3-1-MRC1 plot over
C3-1-CD3E plot over
C3-1-CD8A plot over
C3-1-CD4 plot over
C3-1-FXOP3 plot over
C3-1-BANK1 plot over
C3-1-CD19 plot over
C3-1-MPO plot over
C3-1-IL8 plot over
C3-1-CD94 plot over
C3-1-CD14 plot over
C3-1-CD11c plot over
C3-1-SLA-DRB1 plot over
C3-1-CD31 plot over
C3-1-RTKN2 plot over
C3-1-SFTPB plot over
C3-1 plot over
(1979, 1948)
C3-2-CD163 plot over
C3-2-MRC1 plot over
C3-2-CD3E plot over
C3-2-CD8A plot over
C3-2-CD4 plot over
C3-2-FXOP3 plot over
C3-2-BANK1 plot over
C3-2-CD19 plot over
C3-2-MPO plot over
C3-2-IL8 plot over
C3-2-CD94 plot over
C3-2-CD14 plot over
C3-2-CD11c plot over
C3-2-SLA-DRB1 plot over
C3-2-CD31 plot over
C3-2-RTKN2 plot over
C3-2-SFTPB plot over
C3-2 plot over
(1953, 1946)
C3-3-CD163 plot over
C3-3-MRC1 plot over
C3-3-CD3E plot over
C3-3-CD8A plot over
C3-3-CD4 plot over
C3-3-FXOP3 plot over
C3-3-BANK1 plot over
C3-3-CD19 plot over
C3-3-MPO plot over
C3-3-IL8 plot over
C3-3-CD94 plot over
C3-3-CD14 plot ov

In [13]:
samples = ["A2-1", "A2-2", "A2-3", "A2-4", "A3-1", "A3-2","A3-3", "A4-1","A4-2", "A4-3", "A5-1", "A5-2", "A5-3", "A6-1", "A6-2", "A6-3", "A6-4"]

from skimage import color

dict_color = {'CD163': '#EE5061', 'MRC1': '#EE5061', 'CD3E': '#0072D4', 'CD8A': '#0072D4', 
              'CD4': '#0072D4', 'FXOP3': '#0072D4', 'BANK1': '#C35D14', 'CD19': '#C35D14' , 'MPO': '#FFA7FF',
              'IL8': '#FFA7FF', 'CD94': '#757FAF', 'CD14': '#A70063', 'CD11c': '#F9F871', 'SLA-DRB1' : '#F9F871',
		      'CD31' : '#FF9A6B','RTKN2' : '#296218', 'SFTPB' : '#5ABAB6', 'B646L': '#7F1684',  'MGF_110-9L':'#7F1684',
                'MGF_360-9L':'#7F1684', 'MGF_360-2L':'#7F1684', 'MGF_505-7R':'#7F1684', 'EP402R':'#7F1684', 
              'S273R':'#7F1684'}

def mask2gray(img_path):
    mask = imread(img_path)
    mask=mask[0][0][0][0]
    mask = np.where(mask != 0, 255, 0)
    rgba_image = color.label2rgb(mask, bg_color=(1, 1, 1, 0), bg_label=0)
    gray_image = color.rgb2gray(rgba_image)
    return gray_image
    
for sample in samples:

    os.makedirs(f"./scatterplot/{sample}", exist_ok=True)
    location = f"./{spots_dir}/{sample}-cellspots.csv"
    dir = sample.split('-')[0]
    num = sample.split('-')[1]
    dapi = f"./mask/{dir}_{num}_mask.tif"
    df = pd.read_csv(location)
    img = mask2gray(dapi)
    print(img.shape)
    shape = img.shape[0:2]
    figsize = (shape[1]*0.01, shape[0]*0.01)
    
    #every_gene
    for gene,colors in dict_color.items():
        fig,ax = plt.subplots(figsize=figsize)
        ax.imshow(img, cmap='gray', alpha=0.1)
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=50, c=colors, label=gene)
        plt.legend(prop={"size":100}, loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
        fig.savefig(f"./scatterplot/{sample}/{sample}_{gene}.pdf", format="pdf")
        plt.cla()
        plt.close(fig)
        print(f"{sample}-{gene} plot over")
    
    
    fig_2,ax_2 = plt.subplots(figsize=figsize)
    ax_2.imshow(img, cmap='gray', alpha=0.2)
    for gene,colors in dict_color.items():
        df_gene = df[df["gene"] == gene]
        y = df_gene["dim_1"]
        x = df_gene["dim_2"]
        plt.scatter(x, y, s=35, c=colors, label=gene, alpha=0.70)
    plt.legend(prop={"size":15}, loc=2, ncol = 1, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    #plt.show()
    fig_2.savefig(f"./scatterplot/{sample}-all_gene.pdf", format="pdf")
    plt.cla()
    plt.close(fig_2)
    
    print(f"{sample} plot over")

(1899, 1943)
A2-1-CD163 plot over
A2-1-MRC1 plot over
A2-1-CD3E plot over
A2-1-CD8A plot over
A2-1-CD4 plot over
A2-1-FXOP3 plot over
A2-1-BANK1 plot over
A2-1-CD19 plot over
A2-1-MPO plot over
A2-1-IL8 plot over
A2-1-CD94 plot over
A2-1-CD14 plot over
A2-1-CD11c plot over
A2-1-SLA-DRB1 plot over
A2-1-CD31 plot over
A2-1-RTKN2 plot over
A2-1-SFTPB plot over
A2-1-B646L plot over
A2-1-MGF_110-9L plot over
A2-1-MGF_360-9L plot over
A2-1-MGF_360-2L plot over
A2-1-MGF_505-7R plot over
A2-1-EP402R plot over
A2-1-S273R plot over
A2-1 plot over
(1916, 1948)
A2-2-CD163 plot over
A2-2-MRC1 plot over
A2-2-CD3E plot over
A2-2-CD8A plot over
A2-2-CD4 plot over
A2-2-FXOP3 plot over
A2-2-BANK1 plot over
A2-2-CD19 plot over
A2-2-MPO plot over
A2-2-IL8 plot over
A2-2-CD94 plot over
A2-2-CD14 plot over
A2-2-CD11c plot over
A2-2-SLA-DRB1 plot over
A2-2-CD31 plot over
A2-2-RTKN2 plot over
A2-2-SFTPB plot over
A2-2-B646L plot over
A2-2-MGF_110-9L plot over
A2-2-MGF_360-9L plot over
A2-2-MGF_360-2L plot ove